In [ ]:
import pymongo
import math
import operator

<h2>Main Program</h2>

In [ ]:
connection = pymongo.MongoClient('127.0.0.1',27017)
db = connection['MyTest']
DB_art_wordset = db['art_wordset']
DB_word_dict = db['word_dict']

In [ ]:
# 更新 DB_word_dict 內 IDF 值
N = DB_art_wordset.find().count()
updateIDF(N, DB_word_dict)

In [ ]:
wordsets = DB_art_wordset.find({'keywords':{'$exists':False}})
for wordset in wordsets:
    kw_TFIDF = findkeywords(wordset, DB_word_dict)
    updateKWs(wordset, kw_TFIDF, DB_art_wordset)

<h2>def Functions</h2>

In [ ]:
# 輸入兩 collections, 計算總篇數 N & 更新 word_dict中 IDF值
def updateIDF(N, DB_word_dict):
    word_dict = DB_word_dict.find()
    for word in word_dict:
        idf = math.log(N/word['count'])
        DB_word_dict.update_one({'_id': word['_id']}, {'$set': {'IDF': idf}})
    print('IDF in DB_word_dict updated.')

In [ ]:
# Input一篇文章，與 DB_word_dict比對該文章 wordset裡的字詞
# 計算該文章 wordset的 TFIDF，排序取前 10獲得 keywords dict
def findkeywords(wordset, DB_word_dict):
    tfidf_dict = {}
    for word in wordset['wordset']:
        ref = DB_word_dict.find_one({'_id':word})
        try:
            tfidf = wordset['wordset'][word]['TF']*ref['IDF']
            tfidf_dict[word] = tfidf
        except TypeError:
            DB_art_wordset.find_one_and_update({'_id':wordset['_id']}, {'$unset':{word:1}})
    kw_TFIDF = sorted(tfidf_dict.items(), key=operator.itemgetter(1), reverse=True)[0:10]
    return dict(kw_TFIDF)

In [ ]:
# 新增 DB_art_wordset中該文章 keywords欄位
def updateKWs(wordset, kw_TFIDF, DB_art_wordset):
    DB_art_wordset.update_many({'_id':wordset['_id']}, {'$set':{'keywords':kw_TFIDF}}, upsert=True)
    print('{} is updated keywords!'.format(wordset['_id']))

<h2>LocalHost 本地端測試用</h2>

In [ ]:
# [LocalHost 測試用：建立補齊 DB_word_dict]
def update_DB_word_dict(wordset, DB_word_dict):
    for word in wordset:
        DB_word_dict.update_one({'_id': word}, {'$inc': {'count':1}}, upsert=True)

In [ ]:
# [LocalHost 測試用：建立補齊 DB_word_dict]
articles = DB_news.find()
for article in articles:
    wordset = article['wordset']
    update_DB_word_dict(wordset, DB_word_dict)